# ADES Calling Workspace API Flow

In [ ]:
import requests

domain = "demo.eoepca.org"
auth_server = f"auth.{domain}"
ades_server = f"ades.{domain}"

## Client (e.g. Portal)

In [ ]:
client_id = "e2ca0359-c961-496d-8365-bdaa409b16f3"
client_secret = "9c865400-d976-45db-9929-bba26ba64af1"

### Get Token Endpoint from OIDC Configuration

In [ ]:
# Get OIDC Configuration
oidc_config_endpoint = f"https://{auth_server}/.well-known/openid-configuration"
headers = {"accept": "application/json" }
oidc_config = requests.get(oidc_config_endpoint, headers=headers).json()

# Extract the token endpoint
token_endpoint = oidc_config["token_endpoint"]

### Get Tokens for User `eric`

In [ ]:
eric_name = "eric"
eric_password = "defaultPWD"
headers = { "cache-control": "no-cache" }
data = {
    "scope": "openid user_name is_operator",
    "grant_type": "password",
    "username": eric_name,
    "password": eric_password,
    "client_id": client_id,
    "client_secret": client_secret
}
token_response = requests.post(token_endpoint, headers=headers, data=data).json()
eric_id_token = token_response["id_token"]
eric_access_token = token_response["access_token"]
eric_refresh_token = token_response["refresh_token"]

### Call ADES as user `eric` using the ID token

In [ ]:
# ADES endpoint
ades_eric_endpoint = f"https://{ades_server}/eric/wps3/processes"

In [ ]:
# Request: List Processes
headers = { "accept": "application/json", "Authorization": f"Bearer {eric_id_token}" }
process_list_response = requests.get(ades_eric_endpoint, headers=headers).json()
for process in process_list_response["processes"]:
    print(process["id"])

## ADES as Client

### Receive token from incoming request
Token was createdby/for a different client

In [ ]:
ades_rx_token = eric_id_token
# ades_rx_token = eric_access_token

### Use RX token to call the Workspace API

In [ ]:
# Workspace API endpoint
workspace_api_eric_endpoint = f"https://workspace-api.{domain}/workspaces/demo-user-{eric_name}"

In [ ]:
# Request: Get Workspace Details
headers = { "accept": "application/json", "Authorization": f"Bearer {ades_rx_token}" }
get_eric_workspace_response = requests.get( workspace_api_eric_endpoint, headers=headers ).json()
get_eric_workspace_response

### Create own token via UMA Flow

In [ ]:
# ADES Client Credentials (different from other client)
ades_client_id = "cf394398-5c2e-4580-a67a-6bcc2862f2d6"
ades_client_secret = "70d42655-b1f1-4093-98c1-5d0fd3a23b0a"

Can't demonstrate this with the current endpoints, because the resource-guard (uma-user-agent) abstracts the UMA flow, so we never see the ticket.

In fact, this is exactly what the current resource-guard does - i.e. uses its own client and the provided JWT to get a new access token...
* calls PEP to get a permission ticket
* calls the Token Endpoint with permission ticket + original (received) JWT
* receives access token
* uses access token to confirm authorisation